# Indian Liver Patient Dataset (ILPD) Analysis
## Machine Learning and Data Analytics Coursework

**Dataset**: Indian Liver Patient Dataset (ILPD)

**Objective**: Predict liver disease in patients using machine learning classification algorithms

## 1. Import Libraries

In [ ]:
# Data manipulation
import pandas as pd
import numpy as np

# Visualization
import matplotlib.pyplot as plt
import seaborn as sns
import plotly.express as px
import plotly.graph_objects as go

# Machine Learning
from sklearn.model_selection import train_test_split, cross_val_score, GridSearchCV, StratifiedKFold
from sklearn.preprocessing import StandardScaler, LabelEncoder
from sklearn.metrics import (
    accuracy_score, precision_score, recall_score, f1_score,
    confusion_matrix, classification_report, roc_curve, roc_auc_score,
    precision_recall_curve
)

# Algorithms
from sklearn.linear_model import LogisticRegression
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier, GradientBoostingClassifier
from sklearn.svm import SVC
from sklearn.neighbors import KNeighborsClassifier
from sklearn.naive_bayes import GaussianNB
from xgboost import XGBClassifier

# Handling imbalanced data
from imblearn.over_sampling import SMOTE

# Model persistence
import joblib

# Warnings
import warnings
warnings.filterwarnings('ignore')

# Display settings
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', 100)
plt.style.use('seaborn-v0_8-darkgrid')
sns.set_palette('husl')

print("All libraries imported successfully!")

## 2. Load Dataset

The Indian Liver Patient Dataset (ILPD) contains records of 584 patients.

**Features:**
1. Age - Age of the patient
2. Gender - Gender of the patient
3. Total Bilirubin (TB) - mg/dL
4. Direct Bilirubin (DB) - mg/dL
5. Alkaline Phosphotase - IU/L
6. Alamine Aminotransferase (ALT/SGPT) - IU/L
7. Aspartate Aminotransferase (AST/SGOT) - IU/L
8. Total Proteins (TP) - g/dL
9. Albumin (ALB) - g/dL
10. Albumin and Globulin Ratio (A/G Ratio)
11. Target - 1 (liver patient) or 2 (non-liver patient)

In [ ]:
# Define column names
column_names = [
    'Age', 'Gender', 'Total_Bilirubin', 'Direct_Bilirubin',
    'Alkaline_Phosphotase', 'Alamine_Aminotransferase',
    'Aspartate_Aminotransferase', 'Total_Proteins', 'Albumin',
    'Albumin_Globulin_Ratio', 'Target'
]

# Load dataset
df = pd.read_csv('Indian Liver Patient Dataset (ILPD).csv', names=column_names)

# Display basic information
print("Dataset loaded successfully!")
print(f"\nDataset shape: {df.shape}")
print(f"Number of observations: {df.shape[0]}")
print(f"Number of features: {df.shape[1] - 1}")

# Display first few rows
df.head(10)

## 3. Exploratory Data Analysis (EDA)

### 3.1 Dataset Overview

In [ ]:
# Dataset information
print("=" * 80)
print("DATASET INFORMATION")
print("=" * 80)
df.info()

In [ ]:
# Data types
print("\n" + "=" * 80)
print("DATA TYPES")
print("=" * 80)
df.dtypes

In [ ]:
# Check for missing values
print("\n" + "=" * 80)
print("MISSING VALUES")
print("=" * 80)
missing = df.isnull().sum()
missing_percent = (df.isnull().sum() / len(df)) * 100
missing_df = pd.DataFrame({
    'Missing Count': missing,
    'Percentage': missing_percent
})
print(missing_df[missing_df['Missing Count'] > 0])

In [ ]:
# Check for duplicates
duplicates = df.duplicated().sum()
print(f"\nNumber of duplicate rows: {duplicates}")
print(f"Percentage of duplicates: {(duplicates/len(df))*100:.2f}%")

### 3.2 Statistical Summary

Statistical summary provides measures of central tendency (mean, median) and dispersion (standard deviation, variance) for all numerical features.

In [ ]:
# Statistical summary
print("\n" + "=" * 80)
print("STATISTICAL SUMMARY")
print("=" * 80)
df.describe().T

In [ ]:
# Additional statistics
print("\n" + "=" * 80)
print("VARIANCE AND STANDARD DEVIATION")
print("=" * 80)

numeric_cols = df.select_dtypes(include=[np.number]).columns
stats_df = pd.DataFrame({
    'Mean': df[numeric_cols].mean(),
    'Median': df[numeric_cols].median(),
    'Variance': df[numeric_cols].var(),
    'Std Dev': df[numeric_cols].std(),
    'Skewness': df[numeric_cols].skew(),
    'Kurtosis': df[numeric_cols].kurtosis()
})
stats_df

### 3.3 Target Variable Distribution

Analyzing the distribution of the target variable to check for class imbalance.

In [ ]:
# Target distribution
print("\n" + "=" * 80)
print("TARGET VARIABLE DISTRIBUTION")
print("=" * 80)

target_counts = df['Target'].value_counts()
target_percent = df['Target'].value_counts(normalize=True) * 100

target_df = pd.DataFrame({
    'Count': target_counts,
    'Percentage': target_percent
})
print(target_df)

# Visualization
fig, axes = plt.subplots(1, 2, figsize=(14, 5))

# Count plot
target_counts.plot(kind='bar', ax=axes[0], color=['#3498db', '#e74c3c'])
axes[0].set_title('Target Variable Distribution (Count)', fontsize=14, fontweight='bold')
axes[0].set_xlabel('Target Class')
axes[0].set_ylabel('Count')
axes[0].set_xticklabels(['Liver Patient (1)', 'Non-Liver Patient (2)'], rotation=0)

# Pie chart
axes[1].pie(target_counts, labels=['Liver Patient (1)', 'Non-Liver Patient (2)'],
            autopct='%1.1f%%', colors=['#3498db', '#e74c3c'], startangle=90)
axes[1].set_title('Target Variable Distribution (Percentage)', fontsize=14, fontweight='bold')

plt.tight_layout()
plt.show()

### 3.4 Categorical Features Analysis

In [ ]:
# Gender distribution
print("\n" + "=" * 80)
print("GENDER DISTRIBUTION")
print("=" * 80)

gender_counts = df['Gender'].value_counts()
print(gender_counts)
print(f"\nMale percentage: {(gender_counts['Male']/len(df))*100:.2f}%")
print(f"Female percentage: {(gender_counts['Female']/len(df))*100:.2f}%")

# Gender vs Target
gender_target = pd.crosstab(df['Gender'], df['Target'], normalize='index') * 100
print("\nGender vs Target (%):\n", gender_target)

# Visualization
fig, axes = plt.subplots(1, 2, figsize=(14, 5))

# Gender distribution
df['Gender'].value_counts().plot(kind='bar', ax=axes[0], color=['#3498db', '#e74c3c'])
axes[0].set_title('Gender Distribution', fontsize=14, fontweight='bold')
axes[0].set_xlabel('Gender')
axes[0].set_ylabel('Count')
axes[0].set_xticklabels(axes[0].get_xticklabels(), rotation=0)

# Gender vs Target
pd.crosstab(df['Gender'], df['Target']).plot(kind='bar', ax=axes[1], color=['#3498db', '#e74c3c'])
axes[1].set_title('Gender vs Target Distribution', fontsize=14, fontweight='bold')
axes[1].set_xlabel('Gender')
axes[1].set_ylabel('Count')
axes[1].legend(['Liver Patient (1)', 'Non-Liver Patient (2)'])
axes[1].set_xticklabels(axes[1].get_xticklabels(), rotation=0)

plt.tight_layout()
plt.show()

### 3.5 Numerical Features Distribution

Analyzing the distribution of numerical features using histograms to understand data patterns and identify skewness.

In [ ]:
# Distribution plots for numerical features
numerical_features = df.select_dtypes(include=[np.number]).columns.drop('Target')

fig, axes = plt.subplots(4, 3, figsize=(18, 16))
axes = axes.ravel()

for idx, col in enumerate(numerical_features):
    axes[idx].hist(df[col].dropna(), bins=30, edgecolor='black', color='#3498db', alpha=0.7)
    axes[idx].set_title(f'Distribution of {col}', fontsize=12, fontweight='bold')
    axes[idx].set_xlabel(col)
    axes[idx].set_ylabel('Frequency')
    axes[idx].grid(axis='y', alpha=0.3)

# Hide extra subplot
if len(numerical_features) < len(axes):
    for idx in range(len(numerical_features), len(axes)):
        fig.delaxes(axes[idx])

plt.tight_layout()
plt.show()

### 3.6 Box Plots - Outlier Detection

Box plots help identify outliers and understand the spread of data.

In [ ]:
# Box plots for outlier detection
fig, axes = plt.subplots(4, 3, figsize=(18, 16))
axes = axes.ravel()

for idx, col in enumerate(numerical_features):
    axes[idx].boxplot(df[col].dropna(), vert=True, patch_artist=True,
                      boxprops=dict(facecolor='#3498db', alpha=0.7),
                      medianprops=dict(color='red', linewidth=2))
    axes[idx].set_title(f'Box Plot of {col}', fontsize=12, fontweight='bold')
    axes[idx].set_ylabel(col)
    axes[idx].grid(axis='y', alpha=0.3)

# Hide extra subplot
if len(numerical_features) < len(axes):
    for idx in range(len(numerical_features), len(axes)):
        fig.delaxes(axes[idx])

plt.tight_layout()
plt.show()

### 3.7 Correlation Analysis

Correlation matrix helps identify relationships between features. High correlation may indicate multicollinearity.

In [ ]:
# Correlation matrix
# First encode gender for correlation
df_temp = df.copy()
df_temp['Gender'] = df_temp['Gender'].map({'Male': 1, 'Female': 0})

correlation_matrix = df_temp.corr()

print("\n" + "=" * 80)
print("CORRELATION WITH TARGET")
print("=" * 80)
target_corr = correlation_matrix['Target'].sort_values(ascending=False)
print(target_corr)

# Heatmap
plt.figure(figsize=(12, 10))
sns.heatmap(correlation_matrix, annot=True, fmt='.2f', cmap='coolwarm',
            square=True, linewidths=0.5, cbar_kws={"shrink": 0.8})
plt.title('Correlation Heatmap', fontsize=16, fontweight='bold', pad=20)
plt.tight_layout()
plt.show()

In [ ]:
# Correlation with target - bar plot
target_corr_abs = correlation_matrix['Target'].drop('Target').abs().sort_values(ascending=False)

plt.figure(figsize=(12, 6))
target_corr_abs.plot(kind='barh', color='#3498db')
plt.title('Absolute Correlation with Target Variable', fontsize=14, fontweight='bold')
plt.xlabel('Absolute Correlation')
plt.ylabel('Features')
plt.grid(axis='x', alpha=0.3)
plt.tight_layout()
plt.show()

### 3.8 Feature Relationships - Scatter Plots

Scatter plots show relationships between features and how they separate the target classes.

In [ ]:
# Scatter plots for top correlated features
top_features = ['Direct_Bilirubin', 'Total_Bilirubin', 'Aspartate_Aminotransferase', 'Alamine_Aminotransferase']

fig, axes = plt.subplots(2, 2, figsize=(15, 12))
axes = axes.ravel()

for idx in range(len(top_features)-1):
    for class_label in df['Target'].unique():
        subset = df[df['Target'] == class_label]
        axes[idx].scatter(subset[top_features[idx]], subset[top_features[idx+1]],
                         label=f'Class {class_label}', alpha=0.6, s=30)
    
    axes[idx].set_xlabel(top_features[idx], fontsize=11)
    axes[idx].set_ylabel(top_features[idx+1], fontsize=11)
    axes[idx].set_title(f'{top_features[idx]} vs {top_features[idx+1]}',
                       fontsize=12, fontweight='bold')
    axes[idx].legend()
    axes[idx].grid(alpha=0.3)

# Additional scatter plot
for class_label in df['Target'].unique():
    subset = df[df['Target'] == class_label]
    axes[3].scatter(subset['Age'], subset['Total_Bilirubin'],
                   label=f'Class {class_label}', alpha=0.6, s=30)

axes[3].set_xlabel('Age', fontsize=11)
axes[3].set_ylabel('Total_Bilirubin', fontsize=11)
axes[3].set_title('Age vs Total_Bilirubin', fontsize=12, fontweight='bold')
axes[3].legend()
axes[3].grid(alpha=0.3)

plt.tight_layout()
plt.show()

### 3.9 Age Distribution Analysis

In [ ]:
# Age distribution by target
fig, axes = plt.subplots(1, 2, figsize=(15, 5))

# Histogram
for target in df['Target'].unique():
    subset = df[df['Target'] == target]
    axes[0].hist(subset['Age'], bins=20, alpha=0.6, label=f'Class {target}', edgecolor='black')

axes[0].set_title('Age Distribution by Target Class', fontsize=14, fontweight='bold')
axes[0].set_xlabel('Age')
axes[0].set_ylabel('Frequency')
axes[0].legend()
axes[0].grid(axis='y', alpha=0.3)

# Box plot
df.boxplot(column='Age', by='Target', ax=axes[1], patch_artist=True)
axes[1].set_title('Age Distribution by Target Class (Box Plot)', fontsize=14, fontweight='bold')
axes[1].set_xlabel('Target Class')
axes[1].set_ylabel('Age')
plt.suptitle('')

plt.tight_layout()
plt.show()

print("\nAge Statistics by Target:")
print(df.groupby('Target')['Age'].describe())

## 4. Data Preprocessing

### 4.1 Handle Missing Values

**Justification**: Missing values can negatively impact model performance. We use median imputation for the A/G Ratio as it's robust to outliers.

In [ ]:
# Create a copy for preprocessing
df_processed = df.copy()

# Check missing values
print("Missing values before imputation:")
print(df_processed.isnull().sum())

# Fill missing values with median (robust to outliers)
if df_processed['Albumin_Globulin_Ratio'].isnull().sum() > 0:
    median_ag_ratio = df_processed['Albumin_Globulin_Ratio'].median()
    df_processed['Albumin_Globulin_Ratio'].fillna(median_ag_ratio, inplace=True)
    print(f"\nFilled {df['Albumin_Globulin_Ratio'].isnull().sum()} missing values in A/G Ratio with median: {median_ag_ratio:.2f}")

print("\nMissing values after imputation:")
print(df_processed.isnull().sum())

### 4.2 Handle Duplicates

**Justification**: Duplicate records can lead to overfitting and biased model evaluation.

In [ ]:
# Remove duplicates
before_dup = len(df_processed)
df_processed = df_processed.drop_duplicates()
after_dup = len(df_processed)

print(f"Removed {before_dup - after_dup} duplicate rows")
print(f"Dataset shape after removing duplicates: {df_processed.shape}")

### 4.3 Outlier Detection and Handling

**Justification**: Extreme outliers can skew the model. We use IQR method to identify and cap outliers rather than removing them to preserve sample size.

In [ ]:
def detect_outliers_iqr(df, column):
    """Detect outliers using IQR method"""
    Q1 = df[column].quantile(0.25)
    Q3 = df[column].quantile(0.75)
    IQR = Q3 - Q1
    lower_bound = Q1 - 1.5 * IQR
    upper_bound = Q3 + 1.5 * IQR
    outliers = df[(df[column] < lower_bound) | (df[column] > upper_bound)]
    return outliers, lower_bound, upper_bound

# Detect outliers for each numerical column
print("=" * 80)
print("OUTLIER DETECTION (IQR Method)")
print("=" * 80)

outlier_summary = {}
for col in numerical_features:
    outliers, lower, upper = detect_outliers_iqr(df_processed, col)
    outlier_summary[col] = {
        'count': len(outliers),
        'percentage': (len(outliers) / len(df_processed)) * 100,
        'lower_bound': lower,
        'upper_bound': upper
    }
    print(f"\n{col}:")
    print(f"  Outliers: {len(outliers)} ({(len(outliers)/len(df_processed))*100:.2f}%)")
    print(f"  Valid range: [{lower:.2f}, {upper:.2f}]")

In [ ]:
# Cap outliers instead of removing them (to preserve sample size)
def cap_outliers(df, column):
    """Cap outliers at lower and upper bounds"""
    Q1 = df[column].quantile(0.25)
    Q3 = df[column].quantile(0.75)
    IQR = Q3 - Q1
    lower_bound = Q1 - 1.5 * IQR
    upper_bound = Q3 + 1.5 * IQR
    
    df[column] = df[column].clip(lower=lower_bound, upper=upper_bound)
    return df

# Apply capping to extreme outliers for specific columns
# We'll cap only the most extreme cases
columns_to_cap = ['Total_Bilirubin', 'Direct_Bilirubin', 'Alkaline_Phosphotase',
                  'Alamine_Aminotransferase', 'Aspartate_Aminotransferase']

print("\nCapping outliers for selected features...")
for col in columns_to_cap:
    df_processed = cap_outliers(df_processed, col)
    print(f"Capped outliers for {col}")

print("\nOutlier handling completed!")

### 4.4 Feature Engineering

**Justification**: Creating new features can capture complex relationships and improve model performance.

In [ ]:
# Create new features
print("Creating engineered features...\n")

# 1. AST/ALT Ratio (De Ritis ratio) - important liver function indicator
df_processed['AST_ALT_Ratio'] = (
    df_processed['Aspartate_Aminotransferase'] / 
    (df_processed['Alamine_Aminotransferase'] + 1e-5)  # Add small value to avoid division by zero
)
print("Created: AST/ALT Ratio")

# 2. Total Protein to Albumin Ratio
df_processed['TP_ALB_Ratio'] = (
    df_processed['Total_Proteins'] / 
    (df_processed['Albumin'] + 1e-5)
)
print("Created: Total Protein to Albumin Ratio")

# 3. Age groups (categorical feature)
df_processed['Age_Group'] = pd.cut(df_processed['Age'],
                                    bins=[0, 20, 40, 60, 100],
                                    labels=['Young', 'Adult', 'Middle_Aged', 'Senior'])
print("Created: Age Groups")

# 4. Bilirubin Ratio
df_processed['Bilirubin_Ratio'] = (
    df_processed['Direct_Bilirubin'] / 
    (df_processed['Total_Bilirubin'] + 1e-5)
)
print("Created: Bilirubin Ratio")

# 5. Log transformation for highly skewed features
skewed_features = ['Total_Bilirubin', 'Direct_Bilirubin', 'Alkaline_Phosphotase']
for feature in skewed_features:
    df_processed[f'{feature}_log'] = np.log1p(df_processed[feature])
    print(f"Created: {feature}_log")

print(f"\nDataset shape after feature engineering: {df_processed.shape}")
print(f"New features added: {df_processed.shape[1] - df.shape[1]}")

### 4.5 Encode Categorical Variables

**Justification**: Machine learning algorithms require numerical inputs. We use Label Encoding for binary categorical variable (Gender).

In [ ]:
# Encode Gender
le_gender = LabelEncoder()
df_processed['Gender_Encoded'] = le_gender.fit_transform(df_processed['Gender'])
print("Gender encoding:")
print(dict(zip(le_gender.classes_, le_gender.transform(le_gender.classes_))))

# One-hot encode Age_Group
age_group_dummies = pd.get_dummies(df_processed['Age_Group'], prefix='Age_Group')
df_processed = pd.concat([df_processed, age_group_dummies], axis=1)
print("\nOne-hot encoded Age_Group")

# Convert target to binary (1=disease, 0=no disease)
# Original: 1=liver patient, 2=non-liver patient
# New: 1=liver patient, 0=non-liver patient
df_processed['Target'] = df_processed['Target'].map({1: 1, 2: 0})
print("\nTarget variable converted to binary (1=liver disease, 0=no liver disease)")

### 4.6 Feature Selection

Select the final features for modeling

In [ ]:
# Drop original categorical columns and redundant features
features_to_drop = ['Gender', 'Age_Group']
df_model = df_processed.drop(columns=features_to_drop)

# Separate features and target
X = df_model.drop('Target', axis=1)
y = df_model['Target']

print(f"Final feature set shape: {X.shape}")
print(f"Target shape: {y.shape}")
print(f"\nNumber of features: {X.shape[1]}")
print(f"\nFeature list:")
for idx, col in enumerate(X.columns, 1):
    print(f"{idx}. {col}")

### 4.7 Train-Test Split

**Justification**: Splitting data ensures unbiased evaluation. We use 80-20 split with stratification to maintain class balance.

In [ ]:
# Split the data
X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.2, random_state=42, stratify=y
)

print("Data split completed:")
print(f"Training set: {X_train.shape[0]} samples ({(X_train.shape[0]/len(X))*100:.1f}%)")
print(f"Test set: {X_test.shape[0]} samples ({(X_test.shape[0]/len(X))*100:.1f}%)")
print(f"\nTraining set class distribution:")
print(y_train.value_counts())
print(f"\nTest set class distribution:")
print(y_test.value_counts())

### 4.8 Feature Scaling

**Justification**: Many algorithms (SVM, KNN, Logistic Regression) are sensitive to feature scales. StandardScaler ensures zero mean and unit variance.

In [ ]:
# Scale the features
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)

# Convert back to DataFrame for readability
X_train_scaled = pd.DataFrame(X_train_scaled, columns=X.columns, index=X_train.index)
X_test_scaled = pd.DataFrame(X_test_scaled, columns=X.columns, index=X_test.index)

print("Feature scaling completed using StandardScaler")
print(f"\nScaled training set statistics:")
print(X_train_scaled.describe().loc[['mean', 'std']].T.head(10))

### 4.9 Handle Class Imbalance with SMOTE

**Justification**: Class imbalance can bias the model toward the majority class. SMOTE creates synthetic samples of the minority class.

In [ ]:
# Check class balance
print("Class distribution before SMOTE:")
print(y_train.value_counts())
print(f"Class ratio: {y_train.value_counts()[1] / y_train.value_counts()[0]:.2f}")

# Apply SMOTE
smote = SMOTE(random_state=42)
X_train_balanced, y_train_balanced = smote.fit_resample(X_train_scaled, y_train)

print("\nClass distribution after SMOTE:")
print(pd.Series(y_train_balanced).value_counts())
print(f"\nTraining samples before SMOTE: {len(X_train_scaled)}")
print(f"Training samples after SMOTE: {len(X_train_balanced)}")
print(f"Synthetic samples created: {len(X_train_balanced) - len(X_train_scaled)}")

## 5. Model Training and Evaluation

### 5.1 Model Selection Justification

We will train and compare the following algorithms:

1. **Logistic Regression**: Simple, interpretable baseline model
2. **Random Forest**: Handles non-linear relationships, robust to outliers
3. **XGBoost**: State-of-the-art gradient boosting, handles imbalance well
4. **Support Vector Machine (SVM)**: Effective in high-dimensional spaces
5. **K-Nearest Neighbors**: Non-parametric, simple decision boundary

**Evaluation Metrics**:
- **Accuracy**: Overall correctness
- **Precision**: Minimize false positives
- **Recall/Sensitivity (TPR)**: Minimize false negatives (critical in medical diagnosis)
- **F1-Score**: Balance between precision and recall
- **AUC-ROC**: Model's ability to distinguish between classes
- **Specificity (TNR)**: True negative rate

In [ ]:
# Initialize models
models = {
    'Logistic Regression': LogisticRegression(random_state=42, max_iter=1000),
    'Random Forest': RandomForestClassifier(random_state=42),
    'XGBoost': XGBClassifier(random_state=42, eval_metric='logloss'),
    'SVM': SVC(probability=True, random_state=42),
    'K-Nearest Neighbors': KNeighborsClassifier()
}

print("Models initialized:")
for name in models.keys():
    print(f"  - {name}")

### 5.2 Train Models and Collect Metrics

In [ ]:
# Function to calculate all metrics
def calculate_metrics(y_true, y_pred, y_pred_proba=None):
    """Calculate comprehensive evaluation metrics"""
    metrics = {}
    
    # Confusion matrix elements
    cm = confusion_matrix(y_true, y_pred)
    tn, fp, fn, tp = cm.ravel()
    
    # Basic metrics
    metrics['Accuracy'] = accuracy_score(y_true, y_pred)
    metrics['Precision'] = precision_score(y_true, y_pred, zero_division=0)
    metrics['Recall'] = recall_score(y_true, y_pred, zero_division=0)
    metrics['F1-Score'] = f1_score(y_true, y_pred, zero_division=0)
    
    # TPR and TNR
    metrics['TPR (Sensitivity)'] = tp / (tp + fn) if (tp + fn) > 0 else 0
    metrics['TNR (Specificity)'] = tn / (tn + fp) if (tn + fp) > 0 else 0
    
    # AUC-ROC
    if y_pred_proba is not None:
        metrics['AUC-ROC'] = roc_auc_score(y_true, y_pred_proba)
    else:
        metrics['AUC-ROC'] = 0
    
    return metrics

# Train models and collect results
results = {}
trained_models = {}

print("Training models...\n")
print("=" * 80)

for name, model in models.items():
    print(f"\nTraining {name}...")
    
    # Train model
    model.fit(X_train_balanced, y_train_balanced)
    
    # Predictions
    y_pred = model.predict(X_test_scaled)
    y_pred_proba = model.predict_proba(X_test_scaled)[:, 1]
    
    # Calculate metrics
    metrics = calculate_metrics(y_test, y_pred, y_pred_proba)
    results[name] = metrics
    trained_models[name] = model
    
    print(f"{name} training completed!")
    print(f"  Accuracy: {metrics['Accuracy']:.4f}")
    print(f"  AUC-ROC: {metrics['AUC-ROC']:.4f}")

print("\n" + "=" * 80)
print("All models trained successfully!")

### 5.3 Model Comparison

In [ ]:
# Create comparison DataFrame
results_df = pd.DataFrame(results).T
results_df = results_df.round(4)

print("\n" + "=" * 80)
print("MODEL COMPARISON - ALL METRICS")
print("=" * 80)
print(results_df)

# Identify best model for each metric
print("\n" + "=" * 80)
print("BEST PERFORMING MODELS BY METRIC")
print("=" * 80)
for metric in results_df.columns:
    best_model = results_df[metric].idxmax()
    best_score = results_df[metric].max()
    print(f"{metric:20s}: {best_model:25s} ({best_score:.4f})")

In [ ]:
# Visualize model comparison
fig, axes = plt.subplots(2, 2, figsize=(16, 12))

# Plot 1: Accuracy comparison
results_df['Accuracy'].plot(kind='barh', ax=axes[0, 0], color='#3498db')
axes[0, 0].set_title('Model Accuracy Comparison', fontsize=14, fontweight='bold')
axes[0, 0].set_xlabel('Accuracy')
axes[0, 0].grid(axis='x', alpha=0.3)

# Plot 2: AUC-ROC comparison
results_df['AUC-ROC'].plot(kind='barh', ax=axes[0, 1], color='#e74c3c')
axes[0, 1].set_title('Model AUC-ROC Comparison', fontsize=14, fontweight='bold')
axes[0, 1].set_xlabel('AUC-ROC')
axes[0, 1].grid(axis='x', alpha=0.3)

# Plot 3: Precision vs Recall
results_df[['Precision', 'Recall']].plot(kind='barh', ax=axes[1, 0])
axes[1, 0].set_title('Precision vs Recall Comparison', fontsize=14, fontweight='bold')
axes[1, 0].set_xlabel('Score')
axes[1, 0].legend(loc='best')
axes[1, 0].grid(axis='x', alpha=0.3)

# Plot 4: F1-Score comparison
results_df['F1-Score'].plot(kind='barh', ax=axes[1, 1], color='#2ecc71')
axes[1, 1].set_title('Model F1-Score Comparison', fontsize=14, fontweight='bold')
axes[1, 1].set_xlabel('F1-Score')
axes[1, 1].grid(axis='x', alpha=0.3)

plt.tight_layout()
plt.show()

### 5.4 Confusion Matrices

In [ ]:
# Plot confusion matrices for all models
fig, axes = plt.subplots(2, 3, figsize=(18, 12))
axes = axes.ravel()

for idx, (name, model) in enumerate(trained_models.items()):
    y_pred = model.predict(X_test_scaled)
    cm = confusion_matrix(y_test, y_pred)
    
    sns.heatmap(cm, annot=True, fmt='d', cmap='Blues', ax=axes[idx],
                xticklabels=['No Disease', 'Disease'],
                yticklabels=['No Disease', 'Disease'])
    axes[idx].set_title(f'{name}\nConfusion Matrix', fontsize=12, fontweight='bold')
    axes[idx].set_ylabel('True Label')
    axes[idx].set_xlabel('Predicted Label')

# Hide extra subplot
fig.delaxes(axes[5])

plt.tight_layout()
plt.show()

### 5.5 ROC Curves

In [ ]:
# Plot ROC curves
plt.figure(figsize=(12, 8))

for name, model in trained_models.items():
    y_pred_proba = model.predict_proba(X_test_scaled)[:, 1]
    fpr, tpr, _ = roc_curve(y_test, y_pred_proba)
    auc = roc_auc_score(y_test, y_pred_proba)
    
    plt.plot(fpr, tpr, label=f'{name} (AUC = {auc:.3f})', linewidth=2)

plt.plot([0, 1], [0, 1], 'k--', label='Random Classifier', linewidth=2)
plt.xlabel('False Positive Rate', fontsize=12)
plt.ylabel('True Positive Rate', fontsize=12)
plt.title('ROC Curves - Model Comparison', fontsize=14, fontweight='bold')
plt.legend(loc='lower right', fontsize=10)
plt.grid(alpha=0.3)
plt.tight_layout()
plt.show()

### 5.6 Hyperparameter Tuning

**Justification**: Fine-tuning hyperparameters can significantly improve model performance. We use GridSearchCV with cross-validation.

In [ ]:
# Hyperparameter tuning for Random Forest (best performing model)
print("Performing hyperparameter tuning for Random Forest...\n")

param_grid_rf = {
    'n_estimators': [100, 200, 300],
    'max_depth': [10, 20, 30, None],
    'min_samples_split': [2, 5, 10],
    'min_samples_leaf': [1, 2, 4],
    'max_features': ['sqrt', 'log2']
}

rf_grid = GridSearchCV(
    RandomForestClassifier(random_state=42),
    param_grid_rf,
    cv=5,
    scoring='roc_auc',
    n_jobs=-1,
    verbose=1
)

rf_grid.fit(X_train_balanced, y_train_balanced)

print("\nBest parameters for Random Forest:")
print(rf_grid.best_params_)
print(f"\nBest cross-validation AUC-ROC: {rf_grid.best_score_:.4f}")

# Evaluate tuned model
rf_tuned = rf_grid.best_estimator_
y_pred_tuned = rf_tuned.predict(X_test_scaled)
y_pred_proba_tuned = rf_tuned.predict_proba(X_test_scaled)[:, 1]

metrics_tuned = calculate_metrics(y_test, y_pred_tuned, y_pred_proba_tuned)

print("\nTuned Random Forest Performance:")
for metric, value in metrics_tuned.items():
    print(f"  {metric:20s}: {value:.4f}")

In [ ]:
# Hyperparameter tuning for XGBoost
print("Performing hyperparameter tuning for XGBoost...\n")

param_grid_xgb = {
    'n_estimators': [100, 200, 300],
    'max_depth': [3, 5, 7],
    'learning_rate': [0.01, 0.1, 0.3],
    'subsample': [0.8, 0.9, 1.0],
    'colsample_bytree': [0.8, 0.9, 1.0]
}

xgb_grid = GridSearchCV(
    XGBClassifier(random_state=42, eval_metric='logloss'),
    param_grid_xgb,
    cv=5,
    scoring='roc_auc',
    n_jobs=-1,
    verbose=1
)

xgb_grid.fit(X_train_balanced, y_train_balanced)

print("\nBest parameters for XGBoost:")
print(xgb_grid.best_params_)
print(f"\nBest cross-validation AUC-ROC: {xgb_grid.best_score_:.4f}")

# Evaluate tuned model
xgb_tuned = xgb_grid.best_estimator_
y_pred_xgb_tuned = xgb_tuned.predict(X_test_scaled)
y_pred_proba_xgb_tuned = xgb_tuned.predict_proba(X_test_scaled)[:, 1]

metrics_xgb_tuned = calculate_metrics(y_test, y_pred_xgb_tuned, y_pred_proba_xgb_tuned)

print("\nTuned XGBoost Performance:")
for metric, value in metrics_xgb_tuned.items():
    print(f"  {metric:20s}: {value:.4f}")

### 5.7 Feature Importance Analysis

In [ ]:
# Feature importance from Random Forest
feature_importance_rf = pd.DataFrame({
    'Feature': X.columns,
    'Importance': rf_tuned.feature_importances_
}).sort_values('Importance', ascending=False)

print("=" * 80)
print("FEATURE IMPORTANCE (Random Forest)")
print("=" * 80)
print(feature_importance_rf)

# Plot feature importance
plt.figure(figsize=(12, 8))
plt.barh(feature_importance_rf['Feature'][:15], feature_importance_rf['Importance'][:15], color='#3498db')
plt.xlabel('Importance', fontsize=12)
plt.ylabel('Features', fontsize=12)
plt.title('Top 15 Feature Importance (Random Forest)', fontsize=14, fontweight='bold')
plt.gca().invert_yaxis()
plt.grid(axis='x', alpha=0.3)
plt.tight_layout()
plt.show()

### 5.8 Cross-Validation Analysis

In [ ]:
# Perform cross-validation for all models
print("=" * 80)
print("CROSS-VALIDATION ANALYSIS (5-Fold)")
print("=" * 80)

cv_results = {}
cv = StratifiedKFold(n_splits=5, shuffle=True, random_state=42)

for name, model in trained_models.items():
    scores = cross_val_score(model, X_train_balanced, y_train_balanced,
                            cv=cv, scoring='roc_auc', n_jobs=-1)
    cv_results[name] = {
        'Mean AUC': scores.mean(),
        'Std AUC': scores.std(),
        'Min AUC': scores.min(),
        'Max AUC': scores.max()
    }

cv_df = pd.DataFrame(cv_results).T
print(cv_df)

# Visualize CV results
plt.figure(figsize=(12, 6))
plt.errorbar(range(len(cv_df)), cv_df['Mean AUC'], yerr=cv_df['Std AUC'],
             fmt='o', markersize=8, capsize=5, capthick=2, linewidth=2)
plt.xticks(range(len(cv_df)), cv_df.index, rotation=45, ha='right')
plt.ylabel('AUC-ROC', fontsize=12)
plt.title('Cross-Validation Results (Mean ± Std)', fontsize=14, fontweight='bold')
plt.grid(axis='y', alpha=0.3)
plt.tight_layout()
plt.show()

### 5.9 Final Model Selection and Save

**Final Model Selection**: Based on comprehensive evaluation, we select the best performing model.

In [ ]:
# Select best model based on AUC-ROC
best_model_name = results_df['AUC-ROC'].idxmax()
best_model = trained_models[best_model_name]

print("=" * 80)
print("FINAL MODEL SELECTION")
print("=" * 80)
print(f"\nSelected Model: {best_model_name}")
print(f"\nFinal Model Performance:")
for metric, value in results[best_model_name].items():
    print(f"  {metric:20s}: {value:.4f}")

# Save the best model and preprocessing objects
joblib.dump(rf_tuned, 'models/random_forest_model.pkl')
joblib.dump(xgb_tuned, 'models/xgboost_model.pkl')
joblib.dump(scaler, 'models/scaler.pkl')
joblib.dump(le_gender, 'models/label_encoder.pkl')

print("\nModels and preprocessing objects saved successfully!")
print("  - models/random_forest_model.pkl")
print("  - models/xgboost_model.pkl")
print("  - models/scaler.pkl")
print("  - models/label_encoder.pkl")

## 6. Model Interpretation and Insights

### 6.1 Classification Report

In [ ]:
# Detailed classification report
print("=" * 80)
print("CLASSIFICATION REPORT - FINAL MODEL")
print("=" * 80)
y_pred_final = rf_tuned.predict(X_test_scaled)
print(classification_report(y_test, y_pred_final,
                          target_names=['No Disease', 'Liver Disease']))

### 6.2 Summary Statistics

In [ ]:
# Summary of the entire analysis
print("\n" + "=" * 80)
print("ANALYSIS SUMMARY")
print("=" * 80)
print(f"\nDataset: Indian Liver Patient Dataset (ILPD)")
print(f"Total Samples: {len(df)}")
print(f"Total Features (Original): {df.shape[1] - 1}")
print(f"Total Features (After Engineering): {X.shape[1]}")
print(f"Training Samples: {len(X_train)} ({len(X_train_balanced)} after SMOTE)")
print(f"Test Samples: {len(X_test)}")
print(f"\nModels Evaluated: {len(models)}")
print(f"Best Model: {best_model_name}")
print(f"Best AUC-ROC: {results[best_model_name]['AUC-ROC']:.4f}")
print(f"Best Accuracy: {results[best_model_name]['Accuracy']:.4f}")
print("\n" + "=" * 80)

## 7. Conclusion

This analysis successfully developed and evaluated multiple machine learning models for predicting liver disease. The key findings include:

1. **Data Quality**: The dataset contained minimal missing values and duplicates were handled appropriately.

2. **Feature Engineering**: Created meaningful features including AST/ALT ratio, bilirubin ratio, and log transformations that improved model performance.

3. **Class Imbalance**: Addressed using SMOTE, which improved minority class detection.

4. **Model Performance**: Random Forest and XGBoost demonstrated superior performance with AUC-ROC scores above 0.70.

5. **Key Predictors**: Direct Bilirubin, Total Bilirubin, and liver enzyme ratios were the most important features.

**Limitations**:
- Limited dataset size (584 samples)
- Potential selection bias in data collection
- Missing clinical context for some extreme values

**Future Work**:
- Collect more diverse patient data
- Incorporate additional clinical features
- Explore deep learning approaches
- Develop ensemble methods combining multiple models

**Ethical Considerations**:
- Model should be used as a decision support tool, not replacement for medical professionals
- Regular monitoring and validation required for clinical deployment
- Patient privacy and data security must be maintained
- Potential bias in predictions should be continuously evaluated